In [1]:
# default_exp diffdrive


In [2]:
# hide
from fastcore.all import *


# Differential Drive Chapter

Some code to create and display maps/likelihoods in Chapter 4.


In [3]:
# export
import gtsam
import math
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

## Pinhole Figures

In [4]:
# export
def axes(length=1.5):
    """Add axes to a plotly figure"""
    return [
        go.Scatter3d(x=[0,0], y=[0,0], z=[0,length], mode="lines", line=dict(color="red")),
        go.Scatter3d(x=[0,length], y=[0,0], z=[0,0], mode="lines", line=dict(color="green")),
        go.Scatter3d(x=[0,0], y=[0,length], z=[0,0], mode="lines", line=dict(color="blue")),
        go.Scatter3d(x=[0], y=[0], z=[0], mode="markers", marker=dict(size=3, color="cyan")),
    ]

def plane(Z:float):
    return [
            go.Scatter3d(x=[-1.2, 1.2], y=[-1,-1], z=[Z, Z], mode="lines", line=dict(color="blue")), 
            go.Scatter3d(x=[-1.2, 1.2], y=[ 1, 1], z=[Z, Z], mode="lines", line=dict(color="blue")), 
            go.Scatter3d(x=[-1.2,-1.2], y=[-1, 1], z=[Z, Z], mode="lines", line=dict(color="blue")), 
            go.Scatter3d(x=[ 1.2, 1.2], y=[-1, 1], z=[Z, Z], mode="lines", line=dict(color="blue")), 
    ]

def ray(point3, F, color="orange"):
    X,Y,Z = point3
    x,y = F*X/Z, F*Y/Z
    return go.Scatter3d(x=[X,0,x], y=[Y,0,y], z=[Z,0,F], marker=dict(size=3, color=color))

def show_3d(fig):
    camera = dict(up=dict(x=0, y=-1, z=0), center=dict(x=0, y=0, z=0), eye=dict(x=1.7, y=-0.3, z=-1.1))
    fig.update_layout(scene_camera=camera, margin=dict(t=0, r=0, l=0, b=0), title="Pinhole Camera Model", showlegend=False)
    fig.show()


In [5]:
feet = gtsam.Point3(-3,0,5) # point at the feet of the person, 5 meters in front of camera, 3 meters to the left
head = gtsam.Point3(-3,-2,5) # point at the top of the head (note, Y = *minus* 2 meters)
F = 1 # meter
show_3d(go.Figure(data = plane(-F) + [ray(feet, -F), ray(head, -F)] + axes()))

In [6]:
show_3d(go.Figure(data = plane(F) + [ray(feet, F), ray(head, F)] + axes()))